In [ ]:
function mit18086_multigrid
%MIT18086_MULTIGRID
%    Sets up a 1d Poisson test problem and solves it by multigrid.
%    Method uses twogrid recursively using Gauss-Seidel for smoothing
%    and elimination to solve at coarsest level (n<4).
%    Number of pre- and postsmoothing and coarse grid iteration steps
%    can be prescribed.

% 04/2007 by Benjamin Seibold
%            http://www-math.mit.edu/~seibold/
% Feel free to modify for teaching and learning.
%---------------------------------------------------------------------
levels = 5;                                          % size of problem
nu1 = 2;                           % number of presmoothing iterations
nu2 = 2;                          % number of postsmoothing iterations
gamma = 2;   % number of coarse grid iterations (1=V-cycle, 2=W-cycle)
%---------------------------------------------------------------------
n = 2^(levels+2)-1;                            % number of grid points
h = 1/(n+1);
x = (h:h:(1-h))';
f = pi^2*(sin(pi*x)+4^2*sin(pi*4*x)+9^2*sin(pi*9*x));
A = spdiags(ones(n,1)*[-1 2 -1],-1:1,n,n);
b = f*h^2;
uc = A\b;

global t level, t = 0; level = levels;
clf, subplot(2,2,3:4), hold on
u = twogrid(A,b,nu1,nu2,gamma);
hold off, axis tight
subplot(2,2,1), plot(x,u,'b.-',x,uc,'r.--')
title('correct solution and multigrid approximation')
subplot(2,2,2), plot(x,uc-u,'r.-')
title('error')

%=====================================================================

function x = twogrid(A,b,nu1,nu2,gamma,x0)
%TWOGRID
%    Recursive twogrid cycle for 1d Poisson problem.
%    nu1 = number of presmoothing iterations (Gauss-Seidel)
%    nu2 = number of postsmoothing iterations (Gauss-Seidel)
%    gamma = number of coarse grid iterations (1=V-cycle, 2=W-cycle)
%    x0 = starting vector (0 if not prescribed)
global t level
n = length(b);
if n<4
   x = A\b;                          % solve exactly at coarsest level
else
   G = speye(n)-tril(A)\A; cG = tril(A)\b;       % create Gauss-Seidel
   I = spdiags(ones(n-2,1)*[1 2 1],-2:0,n,n-2); % create interpolation
   I = I(:,1:2:end)/2; R = I'/2;            % and restriction matrices
   if nargin<6, x = b*0; else, x = x0; end           % starting vector
   for i = 1:nu1, x = G*x+cG; end                       % presmoothing
   r = b-A*x;                                       % compute residual
   rh = R*r;                        % restrict residual to coarse grid
   t = t+1; level = level-1; plot([t-1 t], [level+1 level],'bo-')
   eh = rh*0;                                        % starting vector
   for i = 1:gamma
      eh = twogrid(R*A*I,rh,nu1,nu2,gamma,eh); % coarse grid iteration
   end
   e = I*eh;                                       % interpolate error
   t = t+1; level = level+1; plot([t-1 t],[level-1 level],'bo-')
   x = x+e;                                          % update solution
   for i = 1:nu2, x = G*x+cG; end                      % postsmoothing
end


In [3]:
import numpy as np
import warnings
from scipy import sparse
from scipy.sparse import linalg

In [4]:
def twogrid(A, b, nu1, nu2, gamma, x0=None):
    #     TWOGRID
    #         Recursive twogrid cycle for 1d Poisson problem.
    #         nu1 = number of presmoothing iterations (Gauss-Seidel)
    #         nu2 = number of postsmoothing iterations (Gauss-Seidel)
    #         gamma = number of coarse grid iterations (1=V-cycle, 2=W-cycle)
    #         x0 = starting vector (0 if not prescribed)
    warnings.filterwarnings("ignore")
    n = b.size
    if n < 4:
        x = linalg.spsolve(A, b)
    else:
        G = sparse.eye(n) - linalg.inv(sparse.tril(A)).dot(A)
        cG = linalg.inv(sparse.tril(A)).dot(b)
        I = sparse.diags([np.full((n - 2), 0.5), np.ones((n - 2)), 
                        np.full((n - 2), 0.5)], [0, -1, -2], 
                        shape=(n, n - 2), format='csr')
        I = I[:, ::2]
        R = I.T / 2
        if x0 is None:
            x = np.zeros_like(b)
        else:
            x = x0.copy()
        for i in range(nu1):
            x = G.dot(x) + cG
        r = b - A.dot(x)
        rh = R.dot(r)
        eh = np.zeros_like(rh)
        for i in range(gamma):
            eh = twogrid(R.dot(A.dot(I)), rh, nu1, nu2, gamma, eh)
        e = I.dot(eh)
        x += e
        for i in range(nu2):
            x = G.dot(x) + cG
    return x

In [60]:
def twogrid_with_fixes(A, b, nu1, nu2, gamma, x0=None, damp=2./3):
    #     TWOGRID
    #         Recursive twogrid cycle for 1d Poisson problem.
    #         nu1 = number of presmoothing iterations (Gauss-Seidel)
    #         nu2 = number of postsmoothing iterations (Gauss-Seidel)
    #         gamma = number of coarse grid iterations (1=V-cycle, 2=W-cycle)
    #         x0 = starting vector (0 if not prescribed)
    warnings.filterwarnings("ignore")
    n = b.size
    if n < 4:
        x = linalg.spsolve(A, b)
    else:
        I = sparse.diags([np.full((n - 2), 0.5), np.ones((n - 2)), 
                        np.full((n - 2), 0.5)], [0, -1, -2], 
                        shape=(n, n - 2), format='csr')
        I = I[:, ::2]
        R = I.T / 2
        if x0 is None:
            x = np.zeros_like(b)
        else:
            x = x0.copy()
        for i in range(nu1):
            x = damp * 0.5 * (b - A.dot(x) + 2 * x) + (1 - damp) * x
        r = b - A.dot(x)
        rh = R.dot(r)
        eh = np.zeros_like(rh)
        for i in range(gamma):
            print(R.dot(A.dot(I)).shape)
            eh = twogrid(R.dot(A.dot(I)), rh, nu1, nu2, gamma, eh)
        e = I.dot(eh)
        x += e
        for i in range(nu2):
            x = damp * 0.5 * (b - A.dot(x) + 2 * x) + (1 - damp) * x
    return x

In [67]:
levels = 5                  # size of problem
nu1 = 2                     # number of presmoothing iterations
nu2 = 2                     # number of postsmoothing iterations
gamma = 1                   # number of coarse grid iterations (1=V-cycle, 2=W-cycle)
n = 2 ** (levels + 2) - 1   # number of grid points
h = 1.0 / (n + 1)
x = np.linspace(h, 1-h, n)
f = np.ones((n)) #np.pi ** 2 * (np.sin(np.pi * x) + 4 ** 2 * np.sin(np.pi * 4 * x) + 
                 #9 ** 2 * np.sin(np.pi * 9 * x))
A = sparse.diags([np.full((n), 2.), -np.ones((n - 1)), 
              -np.ones((n - 1))], [0, -1, 1], format='csr')
b = f * h ** 2
uc = linalg.spsolve(A, b)
u = np.zeros_like(b)
res_arr = []
for i in range(10):
    u = twogrid_with_fixes(A, b, nu1, nu2, gamma, u)
    res_arr.append(np.linalg.norm(A.dot(u) - b) / np.linalg.norm(b))
res_arr = np.array(res_arr)

(63, 63)
(63, 63)
(63, 63)
(63, 63)
(63, 63)
(63, 63)
(63, 63)
(63, 63)
(63, 63)
(63, 63)


In [68]:
res_arr

array([  1.14642792e-01,   5.07729307e-03,   2.35577902e-04,
         1.26671460e-05,   7.50031243e-07,   4.66157113e-08,
         2.95890424e-09,   1.89228899e-10,   1.21305595e-11,
         8.09033627e-13])

In [69]:
res_arr[1:] / res_arr[:-1]

array([ 0.04428794,  0.04639833,  0.05377052,  0.05921075,  0.06215169,
        0.0634744 ,  0.06395236,  0.06410522,  0.06669384])